
## Prerequisites:

Before running this notebook, make sure you have gone through the steps listed below: 

 - You have a workspace created https://docs.microsoft.com/en-us/azure/machine-learning/service/quickstart-get-started
 <br>
 - You have a development environment configured https://docs.microsoft.com/en-us/azure/machine-learning/service/how-to-configure-environment

In [1]:
%load_ext autoreload
%autoreload 2

import os
from azureml.core import  (Workspace,Run,VERSION, 
                           Experiment,Datastore)
from azureml.core.runconfig import (RunConfiguration,
                                    DEFAULT_GPU_IMAGE)
from azureml.core.conda_dependencies import CondaDependencies
from azureml.core.compute import BatchAiCompute,ComputeTarget,DsvmCompute
from azureml.core.compute_target import ComputeTargetException
from azureml.data.data_reference import DataReference
from azureml.pipeline.core import (Pipeline, 
                                   PipelineData)
from azureml.pipeline.steps import PythonScriptStep
from azureml.train.widgets import RunDetails
import pandas as pd


print("AML SDK version :", VERSION)

AML SDK version : 0.1.74


In [12]:
subscription_id = 'fe375bc2-9f1a-4909-ad0d-9319806d5e97'
resource_group = 'amlenv_rg'
workspace_name = 'vienna'
location = 'westeurope'

In [13]:
project_folder = os.getcwd()
exp_name = "facereco"

ws = Workspace(workspace_name = workspace_name,
               subscription_id = subscription_id,
               resource_group = resource_group)

ws.write_config()
print('Workspace loaded:', ws.name)

Wrote the config file config.json to: /home/sasuke/dev/amlsamples/facereco/aml_config/config.json
Workspace loaded: vienna


## Data Store

Whilst the preprocessed dataset have been made available, you can download from [here](https://amlgitsamples.blob.core.windows.net/facereco/fgnet.zip), upload it over to your Azure blob storage account and point to it in the cell below


In [14]:
account_name = "amlgitsamples"
container_name = "facereco"
datastore_name = 'fgnet'
datastore = Datastore.register_azure_blob_container(workspace = ws, 
                                        datastore_name = datastore_name, 
                                        container_name = container_name,
                                        account_name = account_name, 
                                        overwrite = True)

## Compute target 

Here we choose to execute the pipeline on Batch AI, but you can easily swap the compute target to other [supported types](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-ml-pipelines#key-advantages)

In [23]:
cluster_name = "dlvm"#"batchai-cluster"

try:
    cluster = ComputeTarget(ws, cluster_name)
    print(cluster_name, "found")
    
except ComputeTargetException:
    print(cluster_name, "not found, provisioning....")
    provisioning_config = DsvmCompute.provisioning_configuration(vm_size = "STANDARD_NC6",
                                                                    autoscale_enabled = True,
                                                                    cluster_min_nodes = 2, 
                                                                    cluster_max_nodes = 3)

    
    cluster = ComputeTarget.create(ws, cluster_name, provisioning_config)
    cluster.wait_for_completion(show_output=True)

dlvm found


## Run configuration


Here, we define the conda environment along with the packages dependencies needed by our training scripts along with the [run configuration](https://docs.microsoft.com/en-us/azure/machine-learning/service/concept-azure-machine-learning-architecture#run-configuration).

In [38]:
cd = CondaDependencies()
cd.add_conda_package('pandas')
cd.add_channel(channel = 'menpo')
cd.add_conda_package('matplotlib')
cd.add_conda_package('opencv')
cd.add_conda_package('scikit-learn')

cd.add_pip_package('keras==2.2.0')
cd.add_pip_package('keras-vggface')


run_config = RunConfiguration(framework="python",
                              conda_dependencies= cd)
run_config.target = cluster
run_config.environment.docker.enabled = True
run_config.environment.docker.base_image = DEFAULT_GPU_IMAGE
run_config.environment.python.user_managed_dependencies = False

print(run_config.data_references)

{}


## Pipeline data input/output

We define a reference to the data store we registered earlier that point to the storage which contains the images.

Note that the pipelineData objects uses the default the data store of the workspace.

In [32]:
images_dir = DataReference(data_reference_name = 'images', 
                             path_on_datastore = 'fgnet', 
                             mode ="download", 
                             datastore = datastore
                          )
[print(ds.name) for _,ds in ws.datastores.items()]
default_datastore=ws.datastores["workspaceblobstore"]

print(default_datastore)

metadata_dir = PipelineData(name = 'outputs')
vggface_dir = PipelineData(name = 'vggface')
pca_dir = PipelineData(name = 'pca')
clf_dir = PipelineData(name = 'outputs')

workspaceblobstore
workspacefilestore
fgnet


## Pipeline steps
 

Below are the four steps files that makes up the pipeline. For detailed description of all steps, refer to the readme file [here](https://github.com/Azure/AMLSamples/blob/master/facereco/readme.md).
    
   - Step 1 metadata processing [file](./preprocess.py)
   - Step 2 VGG-Face features extraction [file](./vggface_features.py)
   - Step 3 Dimensionality reduction [file](./pca.py)
   - Step 4 Classifier training [file](./classifier.py)
   
Next, we declare the steps that makes up the pipeline

In [33]:
metadata_processing = PythonScriptStep(
                            name = 'process images metadata',
                            script_name = 'preprocess.py',
                            arguments = ['--images_dir', images_dir,\
                                         '--metadata_path', metadata_dir],
                            inputs = [images_dir],
                            outputs = [metadata_dir],
                            target = cluster_name,
                            runconfig = run_config
                        )


vggface_features = PythonScriptStep(
                            name = 'VGG-face features extractor',
                            script_name = 'vggface_features.py',
                            arguments = ['--metadata_path', metadata_dir,\
                                         '--images_dir', images_dir,\
                                        '--vggface_path', vggface_dir],
                            inputs = [metadata_dir, images_dir],
                            outputs = [vggface_dir],
                            target = cluster_name,
                            runconfig = run_config
                        )

pca_features = PythonScriptStep(
                            name = 'PCA features extractor',
                            script_name = 'pca.py',
                            arguments = ['--vggface_path', vggface_dir,\
                                        '--pca_path', pca_dir],
                            inputs = [vggface_dir],
                            outputs = [pca_dir],
                            target = cluster_name,
                            runconfig = run_config
                        )

classifier_step = PythonScriptStep(
                            name = 'Fit classifier',
                            script_name = 'classifier.py',
                            arguments = ['--vggface_path', vggface_dir,\
                                        '--pca_path', pca_dir,\
                                        '--clf_path', clf_dir],
                            inputs = [vggface_dir, pca_dir],
                            outputs = [clf_dir],
                            target = cluster_name,
                            runconfig = run_config
                        )

## Pipeline execution

Finally we put it all together, construct an experiment and train the pipeline.

In [34]:
pipeline = Pipeline(default_datastore=ws.datastores["workspaceblobstore"],
                description = 'face recognition pipeline', 
                default_source_directory = project_folder,
                workspace = ws,
                steps = [classifier_step]
                   )

pipeline_run = Experiment(workspace=ws, name ="Face_recognition_exp").submit(pipeline, regenerate_outputs=True)
RunDetails(pipeline_run).show()

Created underlying module 1020705a-1d63-4687-b867-6610b93fd3df for StepId [aafe4ec8] - Fit classifier. (This will generate a new run.)
Created underlying module b5e8c7a4-2363-4b85-bb7a-e5a2f2858282 for StepId [30b349a8] - VGG-face features extractor. (This will generate a new run.)
Created underlying module 101fedea-0e60-4ac0-a89d-f8acda09c51c for StepId [13ea2319] - process images metadata. (This will generate a new run.)
Created underlying module fd41703b-7efc-464e-9dc0-f2b4462bc22f for StepId [d1a33a82] - PCA features extractor. (This will generate a new run.)
Created datasource id 841b7a4c-c8e1-4c73-a4af-38c608af12d0 for StepId [61a2497f] - images. (Consumers of this data will generate new runs.)
Created datasource id f3c66d99-488b-4280-86fe-42cde320fa02 for StepId [ebec7b64] - images. (Consumers of this data will generate new runs.)
Submitted pipeline run: 9cc75ade-9ea4-4b19-a27f-2beec8d0868f


_UserRun()

_UserRun(widget_settings={'display': ''})

In [37]:
from azureml.pipeline.core import StepRun

step_run = StepRun(experiment = ws.experiments["Face_recognition_exp"],\
                   pipeline_run_id = "9cc75ade-9ea4-4b19-a27f-2beec8d0868f",\
                   node_id = "13ea2319",
                   step_run_id = "9cc75ade9ea44b19a27f2beec8d0868f_13ea2319_11-23-2018_02-58-38_PM")

joblog = step_run.get_job_log()
print('exec log:', joblog)

stdout_log = step_run.get_stdout_log()
print('stdout log:', stdout_log)

stderr_log = step_run.get_stderr_log()
print('stderr log:', stderr_log)

with open(project_folder + "\\logs-" + step_run.name + ".txt", "w") as f:
    f.write(joblog)
    print("Job log written to logs-"+ step_run.name + ".txt")

exec log: [2018-11-23 14:58:42Z] The job is initialized and Will be submitted to target MLC
[2018-11-23 14:58:43Z] Start Run in Execution Service
[2018-11-23 14:58:45Z] Failed to start the job because of error: Microsoft.RelInfra.Extensions.HttpRequestDetailException: Response status code does not indicate success: 400 (Your runconfig has DataStoreMountConfiguration. However mounting is not supported on this type(Remot).
{
  "error": {
    "code": "UserError",
    "message": "Your runconfig has DataStoreMountConfiguration. However mounting is not supported on this type(Remote) of compute dlvm. Please remove from run config and retry again.",
    "target": null,
    "details": [],
    "innerError": null,
    "debugInfo": {
      "type": "Microsoft.MachineLearning.Common.WebApi.Exceptions.BadRequestException",
      "message": "Your runconfig has DataStoreMountConfiguration. However mounting is not supported on this type(Remote) of compute dlvm. Please remove from run config and retry ag